In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from IPython.display import SVG, Image
#from livelossplot import PlotLossesTensorFlowKeras
from livelossplot import PlotLossesKeras
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.7.0


In [2]:
for expression in os.listdir("final_images/train/"):
    print(str(len(os.listdir("final_images/train/" + expression)))+ " " + expression + " images")

10009 eyebrow_raise images
10370 frown images
10414 smile images
10085 squeezed_eyes images


In [3]:
width = 640
height = 480
batch_size = 8
datagen_train = ImageDataGenerator(rescale=1./255,
                  horizontal_flip=True, 
                  rotation_range=10,
                  width_shift_range=0.1,
                  height_shift_range=0.1,
                  shear_range=0.01,
                  zoom_range=0.01
                )
train_generator = datagen_train.flow_from_directory("final_images/train/",
                  target_size = (width,height),
                  color_mode='grayscale',
                  batch_size=batch_size,
                  class_mode='categorical',shuffle= True)

datagen_validation = ImageDataGenerator(horizontal_flip=True, rescale=1./255)
validation_generator = datagen_validation.flow_from_directory("final_images/validation/",
                  target_size = (width,height),
                  color_mode='grayscale',
                  batch_size=batch_size,
                  class_mode='categorical',shuffle= True)

Found 40878 images belonging to 4 classes.
Found 10037 images belonging to 4 classes.


In [4]:
model= Sequential()
#1 Conv
model.add(Conv2D(8, kernel_size = (3,3), activation = "relu", input_shape=(640,480,1)))
model.add(BatchNormalization())
#model.add(Dropout(0.35))

model.add(Conv2D(16, kernel_size = (3,3), activation = "relu" ))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

model.add(Conv2D(32, kernel_size = (3,3), activation = "relu" ))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size = (3,3), activation = "relu" ))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size = (3,3), activation = "relu" ))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(4, activation='softmax'))

opt=Adam(learning_rate=0.0005)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 638, 478, 8)       80        
                                                                 
 batch_normalization (BatchN  (None, 638, 478, 8)      32        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 636, 476, 16)      1168      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 318, 238, 16)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 316, 236, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 158, 118, 32)     0

In [5]:
epochs = 10
steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps= validation_generator.n//validation_generator.batch_size
checkpoint= ModelCheckpoint("model.h5",monitor='val_accuracy',
                           save_weights_only= False,  
                           mode='max',verbose=2)
callbacks = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

history = model.fit(
          x=train_generator,
          steps_per_epoch=steps_per_epoch,
          epochs =epochs,
          validation_data=validation_generator,
          validation_steps=validation_steps,
          callbacks = callbacks
)

Epoch 1/10
5109/5109 [==============================] - 1781s 348ms/step - loss: 1.4665 - accuracy: 0.2886 - val_loss: 2.5156 - val_accuracy: 0.2637
Epoch 2/10
5109/5109 [==============================] - 1772s 347ms/step - loss: 0.7281 - accuracy: 0.6790 - val_loss: 1.0270 - val_accuracy: 0.6228
Epoch 3/10
5109/5109 [==============================] - 1749s 342ms/step - loss: 0.3561 - accuracy: 0.8835 - val_loss: 0.9841 - val_accuracy: 0.6928
Epoch 4/10
5109/5109 [==============================] - 1733s 339ms/step - loss: 0.2802 - accuracy: 0.9115 - val_loss: 0.8169 - val_accuracy: 0.7156
Epoch 5/10
5109/5109 [==============================] - 1805s 353ms/step - loss: 0.2469 - accuracy: 0.9219 - val_loss: 1.0077 - val_accuracy: 0.6561
Epoch 6/10
5109/5109 [==============================] - 1741s 341ms/step - loss: 0.2243 - accuracy: 0.9288 - val_loss: 0.8877 - val_accuracy: 0.7297
Epoch 7/10
5109/5109 [==============================] - 1743s 341ms/step - loss: 0.2107 - accuracy: 0.9329

In [6]:
test_validation = ImageDataGenerator(horizontal_flip=True, rescale=1./255)

test_generator = test_validation.flow_from_directory("final_images/test/",
                  target_size = (width,height),
                  color_mode='grayscale',
                  batch_size=batch_size,
                  class_mode='categorical',shuffle= False)


STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

test_loss, test_acc = model.evaluate(test_generator)
print('test acc:', test_acc)

Found 10308 images belonging to 4 classes.
1289/1289 [==============================] - 106s 82ms/step - loss: 0.8297 - accuracy: 0.6938
test acc: 0.6938300132751465


In [7]:
predictions = model.predict(test_generator)

In [8]:
predictions

array([[3.4754194e-02, 2.7670765e-01, 2.2028486e-01, 4.6825328e-01],
       [3.4128070e-02, 2.7292335e-01, 2.1178579e-01, 4.8116282e-01],
       [2.8873004e-02, 2.4173012e-01, 9.3213551e-02, 6.3618326e-01],
       ...,
       [2.7648348e-03, 1.5143791e-03, 9.5166284e-01, 4.4057980e-02],
       [4.0662391e-03, 6.5264862e-04, 9.7938341e-01, 1.5897749e-02],
       [1.0355989e-02, 1.5655085e-02, 5.0090694e-01, 4.7308198e-01]],
      dtype=float32)

In [9]:
import numpy as np

predicted_class_indices=np.argmax(predictions,axis=1)

In [10]:
predicted_class_indices

array([3, 3, 3, ..., 2, 2, 2], dtype=int64)

In [11]:
correct_classes = np.array(test_generator.classes)

In [13]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

M = confusion_matrix(correct_classes, predicted_class_indices)
print(M)

[[1733   66  325  467]
 [   1 1355  534  646]
 [   5  354 1837  514]
 [   1  106  115 2249]]


In [14]:
labels = test_generator.class_indices.keys()

In [15]:
import pandas as pd
print("Test")
pd.DataFrame(M, index=labels, columns=labels)

Test


,eyebrow_raise,frown,smile,squeezed_eyes
eyebrow_raise,1733,66,325,467
frown,1,1355,534,646
smile,5,354,1837,514
squeezed_eyes,1,106,115,2249


In [16]:
filepath = r'D:\fakultet\face-emotion-recognition-main\only_high_intensity_69_4%.h5'
model.save(filepath)

In [17]:
from tensorflow import keras
model = keras.models.load_model(filepath)

In [18]:
history2 = model.fit(
          x=train_generator,
          steps_per_epoch=steps_per_epoch,
          epochs =5,
          validation_data=validation_generator,
          validation_steps=validation_steps,
          callbacks = callbacks
)

Epoch 1/5
5109/5109 [==============================] - 1687s 330ms/step - loss: 0.1664 - accuracy: 0.9462 - val_loss: 0.7977 - val_accuracy: 0.7155
Epoch 2/5
5109/5109 [==============================] - 1752s 343ms/step - loss: 0.1571 - accuracy: 0.9483 - val_loss: 2.9083 - val_accuracy: 0.5893
Epoch 3/5
5109/5109 [==============================] - 1750s 342ms/step - loss: 0.1499 - accuracy: 0.9520 - val_loss: 0.7250 - val_accuracy: 0.7138
Epoch 4/5
5109/5109 [==============================] - 1746s 342ms/step - loss: 0.1491 - accuracy: 0.9521 - val_loss: 1.0996 - val_accuracy: 0.7228
Epoch 5/5
5109/5109 [==============================] - 1742s 341ms/step - loss: 0.1391 - accuracy: 0.9547 - val_loss: 0.7844 - val_accuracy: 0.7773


In [19]:
test_validation = ImageDataGenerator(horizontal_flip=True, rescale=1./255)

test_generator = test_validation.flow_from_directory("final_images/test/",
                  target_size = (width,height),
                  color_mode='grayscale',
                  batch_size=batch_size,
                  class_mode='categorical',shuffle= False)


STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

test_loss, test_acc = model.evaluate(test_generator)
print('test acc:', test_acc)

Found 10308 images belonging to 4 classes.
1289/1289 [==============================] - 63s 48ms/step - loss: 0.9244 - accuracy: 0.7238
test acc: 0.7238067388534546


In [20]:
predictions = model.predict(test_generator)
predicted_class_indices=np.argmax(predictions,axis=1)
correct_classes = np.array(test_generator.classes)
M = confusion_matrix(correct_classes, predicted_class_indices)
labels = test_generator.class_indices.keys()
print("Test")
pd.DataFrame(M, index=labels, columns=labels)

Test


,eyebrow_raise,frown,smile,squeezed_eyes
eyebrow_raise,1915,54,342,280
frown,10,1356,479,691
smile,24,346,1965,375
squeezed_eyes,9,130,100,2232


In [21]:
filepath = r'D:\fakultet\face-emotion-recognition-main\only_high_intensity_72_4%.h5'
model.save(filepath)

In [22]:
history3 = model.fit(
          x=train_generator,
          steps_per_epoch=steps_per_epoch,
          epochs =5,
          validation_data=validation_generator,
          validation_steps=validation_steps,
          callbacks = callbacks
)

Epoch 1/5
5109/5109 [==============================] - 2988s 583ms/step - loss: 0.1404 - accuracy: 0.9543 - val_loss: 0.7404 - val_accuracy: 0.7079
Epoch 2/5
5109/5109 [==============================] - 1678s 328ms/step - loss: 0.1396 - accuracy: 0.9543 - val_loss: 3.0858 - val_accuracy: 0.5912
Epoch 3/5
5109/5109 [==============================] - 1686s 330ms/step - loss: 0.1374 - accuracy: 0.9561 - val_loss: 1.4136 - val_accuracy: 0.5981
Epoch 4/5
5109/5109 [==============================] - 1707s 334ms/step - loss: 0.1313 - accuracy: 0.9580 - val_loss: 1.1300 - val_accuracy: 0.6236
Epoch 5/5
5109/5109 [==============================] - 1758s 344ms/step - loss: 0.1234 - accuracy: 0.9593 - val_loss: 1.4795 - val_accuracy: 0.6507


In [23]:
test_validation = ImageDataGenerator(horizontal_flip=True, rescale=1./255)

test_generator = test_validation.flow_from_directory("final_images/test/",
                  target_size = (width,height),
                  color_mode='grayscale',
                  batch_size=batch_size,
                  class_mode='categorical',shuffle= False)


STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

test_loss, test_acc = model.evaluate(test_generator)
print('test acc:', test_acc)

Found 10308 images belonging to 4 classes.
1289/1289 [==============================] - 165s 128ms/step - loss: 1.0481 - accuracy: 0.6321
test acc: 0.6321303844451904
